In [24]:
import pandas as pd
import json
from torch import cuda
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset, concatenate_datasets, DatasetDict
import ast
from transformers import Trainer, EarlyStoppingCallback
from transformers import TrainingArguments
from docx import Document
from docx.shared import RGBColor
from transformers import DataCollatorForTokenClassification
from sklearn.model_selection import KFold
import pickle
from src.models.helpers import *

In [4]:
# label2id = {'O': 0,
#  'I-Treatment': 1,
#  'I-Test': 2,
#  'I-Problem': 3,
#  'I-Background': 4,
#  'I-Other': 5}
# id2label = {id: tag for tag, id in label2id.items()}

# # Print the dictionaries
# print("label2id:", label2id)
# print("id2label:", id2label)

In [5]:
# def transform(example_batch):
#     example_batch['sentence'] = ast.literal_eval(example_batch['sentence'])
#     example_batch['tag'] = ast.literal_eval(example_batch['tag'])
#     example_batch['tag'] = [label2id[label] for label in example_batch['tag']]
#     return example_batch

# def align_labels_with_tokens(labels, word_ids):
#     new_labels = []
#     current_word = None
#     for word_id in word_ids:
#         if word_id != current_word:
#             # Start of a new word!
#             current_word = word_id
#             label = -100 if word_id is None else labels[word_id]
#             new_labels.append(label)
#         elif word_id is None:
#             # Special token
#             new_labels.append(-100)
#         else:
#             # Same word as previous token
#             label = labels[word_id]
#             new_labels.append(label)

#     return new_labels

# def tokenize_and_align_labels(examples, tokenizer):
#     tokenized_inputs = tokenizer(
#         examples["sentence"], truncation=True, is_split_into_words=True
#     )
    
#     all_labels = examples["tag"]
#     new_labels = []
#     for i, labels in enumerate(all_labels):
#         word_ids = tokenized_inputs.word_ids(i)
#         new_labels.append(align_labels_with_tokens(labels, word_ids))

#     tokenized_inputs["labels"] = new_labels
#     return tokenized_inputs

# def tokenize_and_align_labels_wrapper(examples, tokenizer):
#     return tokenize_and_align_labels(examples, tokenizer)

# def untokenize_labels_predictions(word_ids, true_labels, predictions):
#     untokenized_true_labels = []
#     untokenized_predictions = []

#     for sublist_word_ids, sublist_true_labels, sublist_predictions in zip(word_ids, true_labels, predictions):
#         current_labels = []
#         current_predictions = []
#         last_word_id = None

#         for word_id, label, prediction in zip(sublist_word_ids[1:-1], sublist_true_labels, sublist_predictions):
#             # Skip if this word_id is the same as the last one (it's a subword)
#             if word_id == last_word_id:
#                 continue

#             current_labels.append(label)
#             current_predictions.append(prediction)
#             last_word_id = word_id

#         untokenized_true_labels.append(current_labels)
#         untokenized_predictions.append(current_predictions)

#     return untokenized_true_labels, untokenized_predictions

# class TrainingMonitor:
#     def __init__(self):
#         self.best_f1 = 0
#         self.best_confusion_matrix = None

#     def compute_metrics_factory(self, word_ids, fold_no, dataset_name):
#         # Define the actual compute_metrics function
#         def compute_metrics(eval_preds):
#             logits, labels = eval_preds
#             predictions = np.argmax(logits, axis=-1)

#             # Remove ignored index (special tokens) and convert to labels
#             true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
#             true_predictions = [
#                 [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
#                 for prediction, label in zip(predictions, labels)
#             ]

#             untokenized_true_labels, untokenized_predictions = untokenize_labels_predictions(word_ids, true_labels, true_predictions)

#             unflat_true = [label for seq in untokenized_true_labels for label in seq]
#             unflat_pred = [label for seq in untokenized_predictions for label in seq]
#             unreport = classification_report(y_pred=unflat_pred, y_true=unflat_true, output_dict=True)
#             unreport['macro_wo_O'] = {'precision': (unreport['I-Background']['precision'] + unreport['I-Other']['precision'] + unreport['I-Problem']['precision'] + unreport['I-Test']['precision'] + unreport['I-Treatment']['precision']) / 5,
#             'recall': (unreport['I-Background']['recall'] + unreport['I-Other']['recall'] + unreport['I-Problem']['recall'] + unreport['I-Test']['recall'] + unreport['I-Treatment']['recall']) / 5,
#             'f1-score': (unreport['I-Background']['f1-score'] + unreport['I-Other']['f1-score'] + unreport['I-Problem']['f1-score'] + unreport['I-Test']['f1-score'] + unreport['I-Treatment']['f1-score']) / 5,
#             'support': (unreport['I-Background']['support'] + unreport['I-Other']['support'] + unreport['I-Problem']['support'] + unreport['I-Test']['support'] + unreport['I-Treatment']['support'])}
            
#             un_report_df = pd.DataFrame(unreport).round(3).T

#             new_f1_score = unreport['macro_wo_O']['f1-score']
#             if self.best_f1 < new_f1_score:
#                 self.best_f1 = new_f1_score
#                 cm = confusion_matrix(y_pred=unflat_pred, y_true=unflat_true)
#                 disp = ConfusionMatrixDisplay(cm, display_labels=np.array(['I-Background','I-Other', 'I-Problem', 'I-Test', 'I-Treatment', 'O']))
#                 fig, ax = plt.subplots(figsize=(8, 8))
#                 disp.plot(ax=ax)
                

               
#                 binary_predictions = ['0' if label == 'O' else '1' for label in unflat_pred]
#                 binary_labels = ['0' if label == 'O' else '1' for label in unflat_true]

#                 # Generate a classification report
#                 binary_classification_report = classification_report(y_true=binary_labels, y_pred=binary_predictions, target_names=['O', 'I'], digits=3, output_dict=True)
                
#                 # Save the figure to an image file
#                 plt.savefig(f'analysis/{dataset_name}/graphs/fold{fold_no}/confusion_matrix.png')
#                 plt.close()
                
#                 with open(f"analysis/{dataset_name}/reports/fold{fold_no}/multiclass_classification_report.json", "w") as f:
#                     json.dump(un_report_df.to_dict(), f, indent=4)
#                 with open(f"analysis/{dataset_name}/reports/fold{fold_no}/binary_classification_report.json", "w") as f:
#                     json.dump(binary_classification_report, f, indent=4)
                

#             return {
#                 "precision": unreport['macro_wo_O']['precision'],
#                 "recall": unreport['macro_wo_O']['recall'],
#                 "f1": unreport['macro_wo_O']['f1-score'],
#                 "accuracy": unreport['accuracy'],
#             }
#         return compute_metrics



In [114]:
# eval_dataset_name = 'mtsamples2'

model_config = dict(
    train_dataset_name='cross-validation',
    eval_dataset_name = 'cross-validation',
    checkpoint_path = "models/bert-finetuned-ner/fold0/checkpoint-2826", # f'models/models_bert-finetuned-ner_fold0_checkpoint-4710-2024-04-11_11-37-04',
    epochs = 50,
    batch_size = 4,
    learning_rate = 1e-5,
    fold_range = range(0,5),
    output_model_dir = 'models/nooutput'
    )

In [115]:

def train_model(model_config):
    for fold_no in model_config['fold_range']:  
        # Load the tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_config['checkpoint_path'])
        data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

        # Load the model
        model = AutoModelForTokenClassification.from_pretrained(model_config['checkpoint_path'])

        # Load the evaluation data

        # eval_dataset = load_dataset('csv', data_files=f'data/processed/{eval_dataset_name}/final_eval.csv')
        # eval_dataset = eval_dataset.map(transform)
        train_dataset = load_dataset('csv', data_files=f'data/processed/{model_config['train_dataset_name']}/train_fold_{fold_no}.csv')
        eval_dataset = load_dataset('csv', data_files=f'data/processed/{model_config['eval_dataset_name']}/test_fold_{fold_no}.csv')
    
    
        # combined = concatenate_datasets([d1['train'], d2['train']])
        # eval_dataset = DatasetDict({'train': combined})
        train_dataset = train_dataset.map(transform)
        eval_dataset = eval_dataset.map(transform)
        
        
        print(eval_dataset)
        
        # word ids of the tokenized mtsamples to find the original words
        eval_sentences = eval_dataset['train']['sentence']
        eval_word_ids = []
        for sentence in eval_sentences:
            eval_word_ids.append(tokenizer(sentence, truncation=True, is_split_into_words=True).word_ids())
        
        train_sentences = train_dataset['train']['sentence']
        train_word_ids = []
        for sentence in train_sentences:
            train_word_ids.append(tokenizer(sentence, truncation=True, is_split_into_words=True).word_ids())

  

        # Prepare the evaluation dataset
        eval_dataset = eval_dataset.map(lambda examples: tokenize_and_align_labels_wrapper(examples, tokenizer), batched=True, remove_columns=eval_dataset['train'].column_names)
        train_dataset = train_dataset.map(lambda examples: tokenize_and_align_labels_wrapper(examples, tokenizer), batched=True, remove_columns=train_dataset['train'].column_names)

        # split_dataset = eval_dataset['train'].train_test_split(test_size=0.5)


        # Define the training arguments
        training_args = TrainingArguments(
            output_dir=model_config['output_model_dir'], # path of aft-ft
            overwrite_output_dir=True,
            per_device_train_batch_size= model_config['batch_size'],
            per_device_eval_batch_size= model_config['batch_size'],
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            learning_rate=model_config['learning_rate'],
            num_train_epochs=model_config['epochs'],
            logging_strategy="epoch",
        )

        # Initialize the Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset['train'],
            eval_dataset=eval_dataset['train'],
            data_collator=data_collator,
            compute_metrics=TrainingMonitor().compute_metrics_factory(eval_word_ids, fold_no, dataset_name=model_config['eval_dataset_name']),
            callbacks=[EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.01)]
        )

        # trainer.train()
        # Evaluate the model
        
        trainer.evaluate(eval_dataset['train'])
        # print(results)
    

In [116]:
train_model(model_config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


DatasetDict({
    train: Dataset({
        features: ['sentence', 'tag'],
        num_rows: 99
    })
})


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


DatasetDict({
    train: Dataset({
        features: ['sentence', 'tag'],
        num_rows: 99
    })
})


...

DatasetDict({
    train: Dataset({
        features: ['sentence', 'tag'],
        num_rows: 99
    })
})


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


DatasetDict({
    train: Dataset({
        features: ['sentence', 'tag'],
        num_rows: 99
    })
})


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


DatasetDict({
    train: Dataset({
        features: ['sentence', 'tag'],
        num_rows: 99
    })
})


# Model Comparison: 5x2cv

In [103]:
# # generate 5 different train-test data. Shuffle and split 5 times.
# import pandas as pd
# from sklearn.model_selection import ShuffleSplit

# # Let's assume you have a pandas DataFrame called 'df' with your dataset.
# data1 = pd.read_csv('data/processed/mtsamples2/mtsamples_extra.csv')
# data2 = pd.read_csv("data/processed/mtsamples2/final_eval.csv")
# df = pd.concat([data1, data2])


# # Define the ShuffleSplit object to split the data into 2 parts, for 5 splits
# ss = ShuffleSplit(n_splits=5, test_size=0.5, random_state=0)

# # This will shuffle and split the data 5 times
# for fold, (train_index, test_index) in enumerate(ss.split(df)):
#     # Split the data
#     train_set = df.iloc[train_index]
#     test_set = df.iloc[test_index]
    
#     # Save each fold to CSV
#     train_set.to_csv(f'data/processed/cross-validation/train_fold_{fold}.csv', index=False)
#     test_set.to_csv(f'data/processed/cross-validation/test_fold_{fold}.csv', index=False)

In [117]:
# save scores
import json
import os
import numpy as np
import pandas as pd
# This is the base directory where your fold reports are located.
dataset_name = 'cross-validation'
base_dir = f'analysis/{dataset_name}/reports'  
# Initialize dictionaries to hold all the metric values from each fold.
binary_metrics = {
    'O': {'precision': [], 'recall': [], 'f1-score': []},
    'I': {'precision': [], 'recall': [], 'f1-score': []},
    'macro avg': {'precision': [], 'recall': [], 'f1-score': []},
    'weighted avg': {'precision': [], 'recall': [], 'f1-score': []}
}

multiclass_metrics = {
    "macro avg": {'precision': [], 'recall': [], 'f1-score': []},
    "weighted avg": {'precision': [], 'recall': [], 'f1-score': []},
    "macro_wo_O": {'precision': [], 'recall': [], 'f1-score': []}

}

# Process each fold
for fold in range(10):
    fold_dir = os.path.join(base_dir, f'fold{fold}')

    # Load binary classification report
    with open(os.path.join(fold_dir, 'binary_classification_report.json'), 'r') as f:
        binary_report = json.load(f)
        for category in binary_metrics.keys():
            for metric in binary_metrics[category].keys():
                binary_metrics[category][metric].append(binary_report[category][metric])

    # Load multiclass classification report
    multi_df = pd.read_json(os.path.join(fold_dir, 'multiclass_classification_report.json'))
  
    # with open(os.path.join(fold_dir, 'multiclass_classification_report.json'), 'r') as f:
    multiclass_report = multi_df.T.to_dict()
    # print(multiclass_report)

    for category in multiclass_metrics.keys():

        for metric in multiclass_metrics[category].keys():
            multiclass_metrics[category][metric].append(multiclass_report[category][metric])

print(multiclass_metrics['macro_wo_O']['f1-score'])
print(binary_metrics['macro avg']['f1-score'])

# rearrange array
def rearrange_array(array):
    return [x for pair in zip(array[:5], array[5:]) for x in pair]

import pickle
with open('analysis/statistical_test/bc_multiclass_scores.pkl', 'wb') as f:
   pickle.dump(rearrange_array(multiclass_metrics['macro_wo_O']['f1-score']), f)

with open('analysis/statistical_test/bc_binary_scores.pkl', 'wb') as f:
   pickle.dump(rearrange_array(binary_metrics['macro avg']['f1-score']), f)




[0.737, 0.687, 0.678, 0.64, 0.666, 0.618, 0.669, 0.6890000000000001, 0.709, 0.6880000000000001]
[0.8992363744386391, 0.8899396426626842, 0.8975292578505147, 0.888146956137661, 0.8817851765659587, 0.8742837900198306, 0.8822022022650344, 0.8764889889657232, 0.8834996322147446, 0.8904372192730877]


In [118]:
from scipy import stats
def combined_ftest_5x2cv(scores1, scores2):
    """Perform the Combined 5x2CV F-test on two sets of
    model scores to compare their performance.

    Assume R1CV1, R1CV2, R2CV1 ... order
    where R1CV1 is cross-validation repeat 1 and split 1
    Args:
        scores1 (list): Regressor 1 scores on the splits
        scores2 (list): Regressor 2 scores on the splits
    Returns:
        f_stat (float): The F-statistic
        pvalue (float):  If the chosen significance level is larger than
        the p-value, we reject the null hypothesis and accept that
        there are significant differences in the two compared models.
    """
    variances = []
    differences = []
    for i in range(5):
        scores_diff1 = scores1[i * 2] - scores2[i * 2]
        scores_diff2 = scores1[i * 2 + 1] - scores2[i * 2 + 1]
        score_mean = (scores_diff1 + scores_diff2) / 2.0
        score_var = (scores_diff1 - score_mean) ** 2 + \
            (scores_diff2 - score_mean) ** 2

        differences.extend([scores_diff1**2, scores_diff2**2])
        variances.append(score_var)

    numerator = sum(differences)
    denominator = 2 * (sum(variances))
    f_stat = numerator / denominator

    pvalue = stats.f.sf(f_stat, 10, 5)
    return float(f_stat), float(pvalue)

In [119]:
scores1 = pd.read_pickle('analysis/statistical_test/bc_multiclass_scores.pkl')
scores2 = pd.read_pickle('analysis/statistical_test/bc_binary_scores.pkl')

# f_stat, p_val = combined_ftest_5x2cv(scores1, scores2)
# print(f_stat)
# print(p_val)

print("Model 1 mtsamples-ft:", np.mean(scores1), np.std(scores1))
print("Model 2 AFT:", np.mean(scores2), np.std(scores2))


Model 1 mtsamples-ft: 0.6781 0.031643166718898406
Model 2 AFT: 0.8863549240393878 0.007828496371853642
